In [710]:
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.globals import ThemeType,ChartType,SymbolType
from pyecharts.charts import Bar,Pie,WordCloud,Line,Funnel,Polar,Page
from collections import Counter
data_excel = pd.read_excel("药学院大一学生调查.xlsx")

In [711]:
def title():
    title_logo = (
        Pie
        (
            init_opts = opts.InitOpts
            (
                theme=ThemeType.DARK,
                bg_color="#144a74",
                width="1440px",
                height="960px",
                chart_id = "title"
            )
        )
        .set_global_opts
        (
            title_opts = opts.TitleOpts(title = "药学院大一学生认知调查：六景图",pos_left="center"),
            graphic_opts = opts.GraphicGroup
            (
                children = [opts.GraphicImage(graphic_imagestyle_opts=opts.GraphicImageStyleOpts(image ="logo.png" ,width = 40,height = 40))],     #个人logo
             ),
         )
    )
    return title_logo

In [712]:
def pie():
    df = data_excel[['1、您的性别是？', '2、您的生活地是？']]
    df.columns = ['性别', '生活地']
    # 统计四个类别的数量
    categories = [
        ("省内男生", len(df[(df['生活地'] == '广东省内') & (df['性别'] == '男')])),
        ("省内女生", len(df[(df['生活地'] == '广东省内') & (df['性别'] == '女')])),
        ("省外男生", len(df[(df['生活地'] == '广东省外') & (df['性别'] == '男')])),
        ("省外女生", len(df[(df['生活地'] == '广东省外') & (df['性别'] == '女')]))
    ]
    # 创建饼图
    gender_pie = (
        Pie(
            init_opts=opts.InitOpts
            (
                theme=ThemeType.DARK,
                renderer = "svg",
                bg_color="#144a74",
                width="1440px",
                height="960px",
                chart_id = "pie"
            )
        )
        .add(
            "",
            categories,
            radius=["30%", "75%"],
            center=["50%", "50%"],
            rosetype="radius",
            label_opts=opts.LabelOpts(formatter="{b}: {c}人 ({d}%)")
            )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="药学专业学生性别与生源地分布"
            ),
            legend_opts=opts.LegendOpts(
                orient="vertical",
                pos_top="bottom",
                pos_right="2%"
            ),
            tooltip_opts=opts.TooltipOpts(
                trigger="item",
                formatter="{b}: {c}人 ({d}%)"
            )
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(
                font_size=12,
                formatter="{b}: {c} ({d}%)"
            )
        )
    )

    return gender_pie

In [713]:
def cloud():
    df = data_excel["5、为什么来到我们学校选择当前专业？"]
    weekend_activity = [i.split(",") for i in df]
    activity_list = np.unique([i for j in weekend_activity for i in j])
    record_frame = pd.DataFrame(np.zeros((data_excel.shape[0],activity_list.shape[0])), columns = activity_list)
    for i in range(data_excel.shape[0]):
        record_frame.iloc[i,record_frame.columns.get_indexer(weekend_activity[i])] = 1
    record_sum = record_frame.sum(axis = 0)
    activity_name = record_sum.index.tolist()
    activity_count = record_sum.values.tolist()
    activity = [i for i in zip(activity_name, activity_count)]
    reason_cloud = (
        WordCloud
        (
            init_opts=opts.InitOpts
            (
                renderer = "svg",
                theme = ThemeType.DARK,
                bg_color ="#144a74",
                width = "1440px",
                height = "960px",
                chart_id = "cloud"
            )
        )
        .add("",data_pair = activity,shape = "circle")
        .set_global_opts
            (
                legend_opts=opts.LegendOpts(is_show=False),
                title_opts=opts.TitleOpts
                        (
                            title="为什么选此专业",
                        )
            )
    )
    return reason_cloud

In [714]:
def bar():
    df = data_excel.groupby("6、入学前，您想象中的药学专业是怎样的")["6、入学前，您想象中的药学专业是怎样的"].count()
    data_x = df.index.tolist()
    data_y = df.values.tolist()
    impressions_bar = (
        Bar
        (
            init_opts=opts.InitOpts
            (
                theme=ThemeType.DARK,
                renderer = "svg",
                bg_color="#144a74",
                width="1440px",
                height="960px",
                chart_id = "bar"
            )
        )
        .add_xaxis(data_x)
        .add_yaxis("",data_y)
        .set_global_opts
        (
            title_opts = opts.TitleOpts(title="理想中的药学专业"),
            yaxis_opts = opts.AxisOpts(is_show=False),
            xaxis_opts = opts.AxisOpts(splitline_opts = opts.SplitLineOpts()),
            visualmap_opts = opts.VisualMapOpts(is_show = False)
        )
        .set_series_opts(label_opts=opts.LabelOpts(position="top",font_size=20))

    )
    return impressions_bar

In [715]:
def polar():
    data_school = data_excel.groupby("4、您当前的理想职业方向")["4、您当前的理想职业方向"].count()
    data_name = data_school.index.tolist()
    data_value = data_school.values.tolist()

    # 为每个数据项定义不同颜色
    colors = ["#5470C6", "#91CC75", "#FAC858", "#EE6666", "#73C0DE",
              "#3BA272", "#FC8452", "#9A60B4", "#EA7CCC"]

    # 准备带颜色的数据
    colored_data = [
        {"value": val, "name": name, "itemStyle": {"color": colors[i % len(colors)]}}
        for i, (name, val) in enumerate(zip(data_name, data_value))
    ]

    ideal_polar = (
        Polar(
            init_opts=opts.InitOpts(
                renderer="svg",
                theme=ThemeType.DARK,
                bg_color="#144a74",
                width="1440px",
                height="960px",
                chart_id="polar"
            )
        )
        .add_schema(
            angleaxis_opts=opts.AngleAxisOpts(
                is_clockwise=True,
                axistick_opts=opts.AxisTickOpts(is_show=True),
                axislabel_opts=opts.LabelOpts(font_size=20, color="white"),
                axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color="white"))
            ),
            radiusaxis_opts=opts.RadiusAxisOpts(
                data=data_name,
                type_="category",
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=True,font_size=6,rotate=-15)
            )
        )
        .add(
            "",
            colored_data,  # 使用带颜色的数据
            type_="bar",
            symbol="pin",
            symbol_size=10
        )
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_show=False, range_color=["white"]),
            title_opts=opts.TitleOpts(
                title="学生理想职业方向",
                pos_left="left",
                title_textstyle_opts=opts.TextStyleOpts(
                    font_size=18,
                    color="white"
                )
            )
        )
    )
    return ideal_polar

In [716]:
def line():
    df = data_excel.groupby("8、您预期的本科毕业起薪范围")["8、您预期的本科毕业起薪范围"].count()
    data_x = df.index.tolist()
    data_y = df.values.tolist()
    salary_line = (
        Line
        (
            init_opts=opts.InitOpts
            (
                renderer = "svg",
                width = "1440px",
                height = "960px",
                theme = ThemeType.DARK,
                bg_color ="#144a74",
                chart_id = "line"
            )
        )
        .add_xaxis(data_x)
        .add_yaxis
        (
            "",
            y_axis = data_y,
            linestyle_opts = opts.LineStyleOpts(width = 4),
            is_smooth = True
         )
        .set_global_opts
        (
            title_opts=opts.TitleOpts
            (
                title = "预期本科起薪范围"
            ),
            yaxis_opts = opts.AxisOpts(is_show = False),
            xaxis_opts = opts.AxisOpts(splitline_opts = opts.SplitLineOpts()),
            visualmap_opts = opts.VisualMapOpts(is_show = False)
        )
        .set_series_opts(label_opts=opts.LabelOpts(font_size=20))
    )
    return salary_line

In [717]:
def funnel():
    # 提取"9、您未来的优先规划"列并统计频率
    plans = data_excel['9、您未来的优先规划'].dropna().tolist()
    plan_counts = Counter(plans)

    # 按频率排序（从高到低）
    sorted_plans = sorted(plan_counts.items(), key=lambda x: x[1], reverse=True)

    # 准备漏斗图数据
    funnel_data = [(name, count) for name, count in sorted_plans]

    # 创建漏斗图
    future_funnel = (
        Funnel(
            init_opts=opts.InitOpts
            (
                theme=ThemeType.DARK,
                renderer = "svg",
                bg_color="#144a74",
                width="1440px",
                height="960px",
                chart_id = "funnel"
            )
        )
        .add(
            series_name="未来规划",
            data_pair=funnel_data,
            gap=2,
            tooltip_opts=opts.TooltipOpts(trigger="item",formatter="{a} <br/>{b}: {c}人 ({d}%)"),
            label_opts=opts.LabelOpts(is_show=True),
            sort_="ascending",
            itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1)
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="药学专业学生未来规划分布"
            ),
            legend_opts=opts.LegendOpts(is_show=False),
            tooltip_opts=opts.TooltipOpts(trigger="item"),
        )
        .set_series_opts(
            itemstyle_opts=opts.ItemStyleOpts(
                border_width=1,
                border_color="rgba(0,0,0,0.3)"
            )
        )
    )
    return future_funnel

In [718]:
def sign():
    sign_name = (
        Pie
        (
            init_opts = opts.InitOpts
            (
                renderer = "svg",
                theme = ThemeType.DARK,
                bg_color="#144a74",
                width = "1440px",
                height = "960px",
                chart_id = "web_name"
            )
        )
        .set_global_opts
        (
            title_opts = opts.TitleOpts(title = "大数据素养",subtitle = "数据源：药学院大一学生调查.xlsx"),
            graphic_opts = opts.GraphicGroup
            (
                graphic_item=opts.GraphicItem(right = 10,top = 2),
                children = [opts.GraphicImage(graphic_imagestyle_opts=opts.GraphicImageStyleOpts(image = "QR code.png",width = 36,height = 36))],
             )
        )
    )
    return sign_name

In [719]:
def charts():
    six_charts = (
        Page
        (
            layout = Page.DraggablePageLayout,
            page_title = "大学生价值观：六景图",
            #page_border_color = "#144a74"
        )
        .add
        (
            title(),
            pie(),
            cloud(),
            bar(),
            polar(),
            line(),
            funnel(),
            sign()
        )
        .render("SIX_Charts.html")
    )
    return six_charts

In [720]:
if __name__ == "__main__":
    charts()
    Page.save_resize_html("six_charts.html",cfg_file = "chart_config.json",dest = "bigview.html")